#Reference:
https://github.com/rohan-paul/MachineLearning-DeepLearning-Code-for-my-YouTube-Channel/blob/master/NLP/YT_Fine_tuning_BERT_NER_v1.ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class SentenceGetter(object):
    
    def __init__(self, filenames_list):
        self.sentences = []
        self.words = set()
        self.tags = set()
        for filename in filenames_list:
          with open(filename) as f:
              sentence = []
              for line in f:
                  line = line.strip()
                  if (len(line) == 0 or line.startswith("-DOCSTART-") or line.startswith("......")):
                      if len(sentence) != 0:
                        self.sentences.append(sentence)
                        sentence = []
                      continue
                  else:
                      ls = line.split(' ')
                      word, tag = ls[0],ls[3]
                      self.words.add(word)
                      self.tags.add(tag)
                      sentence.append((word,tag))

In [ ]:
#MCHP_dataset_1
files = ['/content/drive/MyDrive/MasterThesis/NER/MCHP/dataset_1/cn569258-pin.conll', '/content/drive/MyDrive/MasterThesis/NER/MCHP/dataset_2/project-13-at-2022-06-14-08-59-1d16c7ec.conll']
getter = SentenceGetter(files)
sentences = getter.sentences
words = getter.words
tags = list(getter.tags)

In [ ]:
tags

['I-Pin', 'O', 'B-Pin']

In [ ]:
from future.utils import iteritems

tag2idx = {t: i for i, t in enumerate(tags)}

idx2tag = {v: k for k, v in iteritems(tag2idx)}

In [ ]:
sentences[0:2]

[[('Appendix', 'O'),
  ('D:', 'O'),
  ('List', 'O'),
  ('of', 'O'),
  ('Tables', 'O'),
  ('Table', 'O'),
  ('2-1:', 'O'),
  ('Pinout', 'O'),
  ('List', 'O')],
 [('Table', 'O'),
  ('3-1:', 'O'),
  ('Power', 'O'),
  ('Down', 'O'),
  ('Scenarios', 'O')]]

In [ ]:
tag2idx

{'I-Pin': 0, 'O': 1, 'B-Pin': 2}

In [ ]:
#sentance tag seperator
def sentence_tag_seperator(sentence_list, tags_dict):
  tokens = []
  tags = []
  for sentence in sentence_list:
    inner_sentence_words = []
    inner_sentence_tags = []
    for word in sentence:
      inner_sentence_words.append(word[0])
      inner_sentence_tags.append(tag2idx[word[1]])
    tokens.append(inner_sentence_words)
    tags.append(inner_sentence_tags)

  return {"tokens" : tokens,
          "tags" : tags}

In [ ]:
total_data = sentence_tag_seperator(sentences,tag2idx)

In [ ]:
total_data.keys()

dict_keys(['tokens', 'tags'])

In [ ]:
# 70,20,10 data split
from sklearn.model_selection import train_test_split
X_data, X_test, Y_data, y_test = train_test_split(total_data['tokens'],total_data['tags'],test_size=0.20,shuffle=True, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_data,Y_data,test_size=0.10,shuffle=True, random_state=42)

train_data = {"tokens" : X_train,"tags" : y_train}
valid_data = {"tokens" : X_val,"tags" : y_val}
test_data = {"tokens" : X_test,"tags" : y_test}


print(
        '\X_tokens length:', len(total_data['tokens']),
        '\ntrain_tokens length:', len(X_train),
        '\nval_tokens length:', len(X_val),
        '\ntest_tokens length:', len(X_test),
        '\n\nY_tags length:', len(total_data['tags']),
        '\ntrain_tags:', len(y_train),
        '\nval_tags:', len(y_val),
        '\ntest_tags:', len(y_test)
    )

\X_tokens length: 545 
train_tokens length: 392 
val_tokens length: 44 
test_tokens length: 109 

Y_tags length: 545 
train_tags: 392 
val_tags: 44 
test_tags: 109


In [ ]:
test_data.keys()

dict_keys(['tokens', 'tags'])

In [ ]:
!pip install transformers datasets tokenizers seqeval -q

In [ ]:
import numpy as np
import datasets 
from transformers import BertTokenizerFast
from transformers import AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
from transformers.data.data_collator import DataCollatorForTokenClassification

In [ ]:
# Transfer Learning Approach
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/MasterThesis/Bert/conll_ner_model", num_labels = 3, ignore_mismatched_sizes=True)
tokenizer = BertTokenizerFast.from_pretrained("/content/drive/MyDrive/MasterThesis/Bert/conll_ner_tokenizer")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at /content/drive/MyDrive/MasterThesis/Bert/conll_ner_model and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model_fine_tuned)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
def tokenize_and_align_labels(example_data, label_all_tokens = True):
  tokenized_inputs = tokenizer(example_data['tokens'], is_split_into_words=True, truncation=True)
  labels = []

  for i, lable in enumerate(example_data['tags']):
    word_ids = tokenized_inputs.word_ids(batch_index=i)

    previous_word_idx = None

    label_ids = []

    for word_idx in word_ids:
      if word_idx is None:
        label_ids.append(-100)
      elif word_idx != previous_word_idx:
        label_ids.append(lable[word_idx])
      else:
        label_ids.append(lable[word_idx] if label_all_tokens else -100)
      
      previous_word_idx = word_idx
    labels.append(label_ids)
  tokenized_inputs["labels"] = labels
  tokenized_inputs["tokens"] = example_data['tokens']
  tokenized_inputs["tags"] = example_data['tags']
  return tokenized_inputs

In [ ]:
#Final tokenized train data
tokenized_dataset_train = tokenize_and_align_labels(train_data)
#convert to apache arrow Datasets to train the model
tokenized_dataset_train = datasets.Dataset.from_dict(tokenized_dataset_train)

In [ ]:
#Final tokenized valid data
tokenized_dataset_valid = tokenize_and_align_labels(valid_data)
tokenized_dataset_valid = datasets.Dataset.from_dict(tokenized_dataset_valid)

In [ ]:
#Final tokenized test data
tokenized_dataset_test = tokenize_and_align_labels(test_data)
tokenized_dataset_test = datasets.Dataset.from_dict(tokenized_dataset_test)

In [ ]:
tokenized_dataset_test

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'tokens', 'tags'],
    num_rows: 109
})

In [ ]:
args = TrainingArguments(
    '/content/drive/MyDrive/MasterThesis/Bert/PIN/test-ner',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    do_train = True,
    do_predict=True
)

In [ ]:
args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignor

In [ ]:
from transformers.data.data_collator import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = datasets.load_metric("seqeval") 

<ipython-input-28-80ed98bfe56d>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval")


In [ ]:
def compute_metrics(eval_preds): 
    pred_logits, labels = eval_preds 
    
    pred_logits = np.argmax(pred_logits, axis=2) 
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax
    
    # We remove all the values where the label is -100
    predictions = [ 
        [tags[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100] 
        for prediction, label in zip(pred_logits, labels) 
    ] 
    
    true_labels = [ 
      [tags[l] for (eval_preds, l) in zip(prediction, label) if l != -100] 
       for prediction, label in zip(pred_logits, labels) 
   ] 
    results = metric.compute(predictions=predictions, references=true_labels) 
    return { 
   "precision": results["overall_precision"], 
   "recall": results["overall_recall"], 
   "f1": results["overall_f1"], 
  "accuracy": results["overall_accuracy"], 
  } 

In [ ]:
trainer = Trainer( 
  model_fine_tuned, 
  args, 
  train_dataset=tokenized_dataset_train, 
  eval_dataset=tokenized_dataset_valid, 
  data_collator=data_collator, 
  tokenizer=tokenizer, 
  compute_metrics=compute_metrics 
) 

In [ ]:
trainer.train() #Epoch 3

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.071830,0.750000,0.021818,0.042403,0.977367
2,No log,0.063646,0.462830,0.701818,0.557803,0.974475
3,No log,0.054148,0.566563,0.665455,0.612040,0.980588


TrainOutput(global_step=75, training_loss=0.09500116348266602, metrics={'train_runtime': 111.1917, 'train_samples_per_second': 10.576, 'train_steps_per_second': 0.675, 'total_flos': 289911372645888.0, 'train_loss': 0.09500116348266602, 'epoch': 3.0})

In [ ]:
predictions, label_ids, metrics  = trainer.predict(test_dataset = tokenized_dataset_test) #epoch 3
metrics

{'test_loss': 0.03675864264369011,
 'test_precision': 0.6127659574468085,
 'test_recall': 0.8212927756653993,
 'test_f1': 0.701868399675061,
 'test_accuracy': 0.986251592118079,
 'test_runtime': 4.844,
 'test_samples_per_second': 22.502,
 'test_steps_per_second': 1.445}

In [ ]:
i = np.random.randint(0,tokenized_dataset_test.shape[0]) # choose a random number between 0 and len(X_te)
p, l, m = trainer.predict([tokenized_dataset_test[i]])
p = np.argmax(p, axis=-1)

# true = np.argmax(tokenized_dataset_test[i]['tags'], -1)
true = tokenized_dataset_test[i]['tags']

print("Sample number {} of {} (Test Set)".format(i, tokenized_dataset_test.shape[0]))
# Visualization
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")

for w, t, pred in zip(tokenized_dataset_test[i]['tokens'], true, p[0][1:(len(p[0])-1)]):
    if w != 0:
        print("{:15}: {:5} {}".format(w, idx2tag[t], idx2tag[pred]))

***** Running Prediction *****
  Num examples = 1
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tags, tokens. If tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.


Sample number 102 of 109 (Test Set)
Word           ||True ||Pred
The            : O     O
EHC            : O     O
should         : O     O
be             : O     O
able           : O     O
to             : O     O
recognize      : O     O
this           : O     O
condition      : O     O
since          : O     O
the            : O     O
RXD            : B-Pin O
pin            : O     O
will           : O     O
be             : O     O
con-           : O     O
stantly        : O     O
high           : O     O
while          : O     O
the            : O     O
TXD            : B-Pin O
pin            : O     O
is             : O     O
sending        : O     O
out            : O     O
data           : O     O
(during        : O     O
normal         : O     O
operation      : O     O
RXD            : B-Pin O
should         : O     O
at             : O     O
least          : O     O
track          : O     O
all            : O     O
TXD            : B-Pin O
low            : O     O
levels).  